<a href="https://colab.research.google.com/github/nganmeni/Depot/blob/main/AnalyseSpatiale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/eco2mix-regional-cons-def.csv', sep=';');

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,54,56,58,60,62,64) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [75]:
regions = df['Région'].unique().tolist()
print(len(regions))
for i in range(12):
  print(regions[i])

12
Pays de la Loire
Bourgogne-Franche-Comté
Normandie
Grand Est
Centre-Val de Loire
Occitanie
Île-de-France
Bretagne
Auvergne-Rhône-Alpes
Hauts-de-France
Provence-Alpes-Côte d'Azur
Nouvelle-Aquitaine


In [39]:
columns = ['Région',
           'Date',
           'Date - Heure',
           'Consommation (MW)',
            'Thermique (MW)',
            'Nucléaire (MW)',
            'Eolien (MW)',
            'Solaire (MW)',
            'Hydraulique (MW)',
            'Bioénergies (MW)']
df_temp = pd.DataFrame(columns=columns)
for i in range(len(columns)): 
  df_temp[columns[i]] = df[columns[i]]
df = df_temp.copy()
df  = df.dropna(how='all', axis=0)
for c in columns[3:]:
    df[c].fillna(0, inplace=True)

In [40]:
df.head()

,Région,Date,Date - Heure,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Bioénergies (MW)
0,Pays de la Loire,2013-01-01,2013-01-01T00:00:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Bourgogne-Franche-Comté,2013-01-01,2013-01-01T00:00:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Normandie,2013-01-01,2013-01-01T00:00:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Grand Est,2013-01-01,2013-01-01T00:00:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Centre-Val de Loire,2013-01-01,2013-01-01T00:00:00+01:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
df_reg = df.groupby(['Région']).aggregate({'Consommation (MW)': 'mean', 
                                                    'Thermique (MW)': 'mean',
                                                    'Nucléaire (MW)': 'mean',
                                                    'Eolien (MW)': 'mean',
                                                    'Solaire (MW)': 'mean',
                                                    'Hydraulique (MW)': 'mean',
                                                    'Bioénergies (MW)': 'mean' }).reset_index()

In [45]:
df_reg.head(14)

,Région,Consommation (MW),Thermique (MW),Nucléaire (MW),Eolien (MW),Solaire (MW),Hydraulique (MW),Bioénergies (MW)
0,Auvergne-Rhône-Alpes,7540.857289,229.901836,9473.350978,110.105879,108.014234,3245.023727,104.465853
1,Bourgogne-Franche-Comté,2440.701707,83.943992,0.000000,124.839631,30.642161,95.935446,25.728005
2,Bretagne,2553.196403,75.209414,0.000000,194.435117,24.188420,66.235699,38.499329
3,Centre-Val de Loire,2155.208508,39.999335,8371.337805,241.150869,32.905226,13.934418,51.377968
4,Grand Est,5206.109464,1031.273465,8827.644215,673.984350,73.138052,952.655622,89.831044
5,Hauts-de-France,5789.643530,912.948719,3835.867125,749.398828,16.684546,1.405562,106.548045
6,Normandie,3205.051355,450.722594,6745.940609,161.408341,16.396323,14.247294,52.137454
7,Nouvelle-Aquitaine,4953.064897,100.108443,4875.430209,162.128693,272.050536,425.748671,161.677046
8,Occitanie,4272.224662,33.245052,2008.147183,335.099393,225.161598,1333.095512,80.740211
9,Pays de la Loire,3096.431807,504.994716,0.000000,175.116386,57.021028,2.068677,42.981765


In [123]:
import folium
France = folium.Map(location = [47.4846513699, 1.68420250928], zoom_start = 5)

In [68]:
import json
geo = json.load(open("/content/drive/MyDrive/Colab Notebooks/georef-france-region.geojson"))

In [69]:
geo.keys()

dict_keys(['type', 'features'])

In [70]:
len(geo["features"])

13

In [71]:
del geo["features"][6]


In [72]:
for reg in geo["features"]:
  prop = reg["properties"]
  print(prop["reg_name"])

Auvergne-Rhône-Alpes
Nouvelle-Aquitaine
Hauts-de-France
Centre-Val de Loire
Île-de-France
Bretagne
Normandie
Pays de la Loire
Grand Est
Occitanie
Provence-Alpes-Côte d'Azur
Bourgogne-Franche-Comté


In [ ]:
geo["features"][0]['geometry']

In [ ]:
France_reg_marker = folium.Map(location = [47.4846513699, 1.68420250928], zoom_start = 5)
for reg in geo["features"]:
    prop = reg["properties"]
    folium.Marker(prop["geo_point_2d"], popup = prop["reg_name"]).add_to(France_reg_marker)
France_reg_marker

In [79]:
df_reg['Nucléaire (MW)']['Auvergne-Rhône-Alpes']

0     9473.350978
1        0.000000
2        0.000000
3     8371.337805
4     8827.644215
5     3835.867125
6     6745.940609
7     4875.430209
8     2008.147183
9        0.000000
10       0.000000
11       0.000000
Name: Nucléaire (MW), dtype: float64

In [121]:
f = folium.Figure(width=800, height=700)
France_reg_circle = folium.Map(location= [47, 2], zoom_start=6, min_zoom = 6,  max_zoom = 7, max_bounds=True ).add_to(f)
#France_reg_circle = folium.Map(location=[48.856578, 2.351828],max_bounds=True)
#France_reg_circle = folium.Map(location = [48.856578, 2.351828], zoom_start = 5)
for arr in geo["features"]:
    prop = arr["properties"]
    folium.CircleMarker(prop["geo_point_2d"], 
                        popup = prop["reg_name"],
                        color="Blue",
                        fill_color="Blue",
                        fill_opacity = 0.8,
                        radius = int(df_reg[df_reg['Région']==prop["reg_name"]]['Nucléaire (MW)']) / 1000).add_to(France_reg_circle)
France_reg_circle

In [124]:

import folium
from IPython.core.display import HTML

m = folium.Map(location=[45.5236, -122.6750])
HTML(m._repr_html_())